In [24]:
import pandas as pd
import numpy as np

In [25]:
def evaluate_strategy(df, target_col, prob_col, balance_col, spend_col, threshold):
   
    df = df.copy()

    # Step 1: Apply strategy
    df['accepted'] = df[prob_col] < threshold
    accepted = df[df['accepted']]

    # Step 2: Calculate default rate among accepted
    default_rate = accepted[target_col].mean() if not accepted.empty else 0

    # Step 3: Calculate monthly and annual revenue
    accepted['monthly_revenue'] = accepted[balance_col] * 0.02 + accepted[spend_col] * 0.001

    # Step 4: Set revenue to zero for defaulters
    accepted['expected_12mo_revenue'] = accepted.apply(
        lambda row: 0 if row[target_col] == 1 else row['monthly_revenue'] * 12,
        axis=1
    )

    total_revenue = accepted['expected_12mo_revenue'].sum()

    return round(default_rate, 4), round(total_revenue,2)

In [26]:
train_df = pd.read_csv('train_filt_label_data.csv')

test1_df = pd.read_csv('xgBoost_test1_pred.csv')

test2_df = pd.read_csv('xgBoost_test2_pred.csv')

In [27]:
train_pred = pd.read_csv('xgBoost_Pred.csv')[['customer_ID','scores','target']]

test1_pred = test1_df[['customer_ID','scores','target']]

test2_pred = test2_df[['customer_ID','scores','target']]

In [28]:
# train_pred = pd.concat([train_pred, test1_pred, test2_pred], ignore_index=True)

In [29]:
train_pred

,customer_ID,scores,target
0,53a5cf7b0a2674e1a1eed00afd7fc34e91bb4592a13f78...,0.041164,0
1,58ef292a9555855528e41447a5d1a654ddd256066452aa...,0.286190,0
2,a25ec3efe7ba472aaf4522a6162e8e26078fad632ea161...,0.003302,0
3,3f3bd3c55b573eac6c9035c508213162da2ee926aa6c63...,0.000601,0
4,b974d50eea3e2a71269c23f01e84f86ef7ae3517b235df...,0.876593,1
...,...,...,...
64243,c5bdd9e1b4b96ec705e153de2b6b7e0d90401654544fc5...,0.774545,1
64244,edb9381ad4c294e59d35bc8bc5f856dc434fa149da8ed2...,0.946615,1
64245,de3f649eaabd612611a120049faefab702479375f58188...,0.000401,0
64246,2d3c2d6fd0a3032ce8213bd28c1c28102a1c89493e8bf8...,0.011398,0


In [30]:

# Step 2: Apply threshold (e.g. 0.5)
threshold = 0.5
train_pred ['predicted_default'] = (train_pred ['scores'] >= threshold).astype(int)

# Step 3: Keep only desired columns
predictions_df = train_pred[['predicted_default']]

In [31]:
train_pred

,customer_ID,scores,target,predicted_default
0,53a5cf7b0a2674e1a1eed00afd7fc34e91bb4592a13f78...,0.041164,0,0
1,58ef292a9555855528e41447a5d1a654ddd256066452aa...,0.286190,0,0
2,a25ec3efe7ba472aaf4522a6162e8e26078fad632ea161...,0.003302,0,0
3,3f3bd3c55b573eac6c9035c508213162da2ee926aa6c63...,0.000601,0,0
4,b974d50eea3e2a71269c23f01e84f86ef7ae3517b235df...,0.876593,1,1
...,...,...,...,...
64243,c5bdd9e1b4b96ec705e153de2b6b7e0d90401654544fc5...,0.774545,1,1
64244,edb9381ad4c294e59d35bc8bc5f856dc434fa149da8ed2...,0.946615,1,1
64245,de3f649eaabd612611a120049faefab702479375f58188...,0.000401,0,0
64246,2d3c2d6fd0a3032ce8213bd28c1c28102a1c89493e8bf8...,0.011398,0,0


In [32]:
train_df['scores'] = train_pred['scores']

In [33]:
train_df

,Unnamed: 0,customer_ID,target,S_2,P_2,D_39,B_1,B_2,R_1,S_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,scores
0,0,12c761e40348fa242de0356426fa9547f0feea34d1f57b...,0,2017-03-21,0.628534,0.004250,0.017667,0.815468,0.005736,0.265531,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.041164
1,1,12c761e40348fa242de0356426fa9547f0feea34d1f57b...,0,2017-04-21,0.669914,0.418321,0.354284,1.000587,0.009774,0.176196,...,NaN,NaN,0.003869,0.008332,0.005805,NaN,0.006603,0.007874,0.002185,0.286190
2,2,12c761e40348fa242de0356426fa9547f0feea34d1f57b...,0,2017-05-08,0.661933,0.003989,0.244867,1.008683,0.005743,0.169471,...,NaN,NaN,0.004872,0.008471,0.006763,NaN,0.009782,0.007285,0.000121,0.003302
3,3,12c761e40348fa242de0356426fa9547f0feea34d1f57b...,0,2017-06-14,0.644270,0.009351,0.017799,0.668398,0.003148,0.174020,...,NaN,NaN,0.009246,0.009905,0.008673,NaN,0.003927,0.001159,0.000433,0.000601
4,4,12c761e40348fa242de0356426fa9547f0feea34d1f57b...,0,2017-07-26,0.688055,0.002832,0.009458,0.779692,0.000870,0.174547,...,NaN,NaN,0.004311,0.006488,0.004953,NaN,0.009283,0.003769,0.006124,0.876593
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1107077,1107077,f0defebbf6e79c14234a06c5b3522ee5914cb0f23fdd13...,0,2017-11-29,0.957207,0.003413,0.005511,0.811611,0.007821,NaN,...,NaN,NaN,0.003099,0.000731,0.006184,NaN,0.006376,0.003519,0.007407,NaN
1107078,1107078,f0defebbf6e79c14234a06c5b3522ee5914cb0f23fdd13...,0,2017-12-30,0.826494,0.007873,0.009774,0.819318,0.007074,NaN,...,NaN,NaN,0.007693,0.001338,0.001450,NaN,0.003571,0.001993,0.005287,NaN
1107079,1107079,f0defebbf6e79c14234a06c5b3522ee5914cb0f23fdd13...,0,2018-01-30,0.828001,0.002666,0.001569,0.815946,0.001636,NaN,...,NaN,NaN,NaN,0.006918,NaN,NaN,NaN,0.001155,NaN,NaN
1107080,1107080,f0defebbf6e79c14234a06c5b3522ee5914cb0f23fdd13...,0,2018-02-27,0.825479,0.001338,0.006197,0.812589,0.000352,NaN,...,NaN,NaN,0.009021,0.006564,0.000960,NaN,0.001825,0.006149,0.009502,NaN


In [112]:
spend_feature = 'S_5'
balance_feature='B_2'

In [113]:
train_df.rename(columns={'S_2': 'date'}, inplace=True)

In [114]:
six_months_mask = (train_df['date'] >= '2017-11-01') & (train_df['date'] <= '2018-04-30')
df_filtered = train_df.loc[six_months_mask, ['customer_ID', 'date', spend_feature, balance_feature]]

In [115]:
avg_df = df_filtered.groupby('customer_ID')[[spend_feature, balance_feature]].mean().reset_index()
avg_df.rename(columns={spend_feature: 'S_Ave', balance_feature: 'B_Ave'},inplace=True)

In [116]:
avg_df['monthly_revenue'] = avg_df['B_Ave'] * 0.02 + avg_df['S_Ave'] * 0.001
avg_df['expected_revenue_12mo'] = avg_df['monthly_revenue']*12

In [117]:
# Merge with your model predictions: predictions_df with ['customer_ID', 'predicted_default']
revenue_df = avg_df.merge(train_pred, on='customer_ID', how='left')

# Apply 0 revenue for predicted defaulters
revenue_df['expected_revenue_12mo'] = revenue_df.apply(
    lambda row: 0 if row['predicted_default'] == 1 else row['expected_revenue_12mo'],
axis=1
)

In [118]:
test1_pred

,customer_ID,scores,target
0,d128fc1905875ec9dbe9991308f5a0ee736a9c424f59d5...,0.401877,1
1,ed4aa1b3781eca4e5674d96baff5575eea4564305bfdc7...,0.635110,0
2,479a03922bed6934e4e986c0f732b7f64433569e02a024...,0.965461,1
3,c5a88d46c3cd1116351363f771b396219d2e9a24e92a2d...,0.004398,0
4,4054a9be2c950c2bcfeec5599777185071c7612938acfe...,0.000527,0
...,...,...,...
13762,a53001c8677c9cc8326b2a9dc2b0d10c87fab0d87323b5...,0.227563,0
13763,bf88337b771dfa9ec7eaf9010ae4c4d6eb2f5a6fd48d5b...,0.000833,0
13764,2e09213ddb96da7032f72856fbce6c92b0e03506480d06...,0.999112,1
13765,4b8c2e72963cc7bb512d986f2c60449d9df8abd2462240...,0.169878,0


In [119]:
# Merge with your model predictions: predictions_df with ['customer_ID', 'predicted_default']
revenue_df_test_1 = avg_df.merge(test1_pred, on='customer_ID', how='left')

# Apply 0 revenue for predicted defaulters
revenue_df_test_1['expected_revenue_12mo'] = revenue_df_test_1.apply(
    lambda row: 0 if row['scores'] == 1 else row['expected_revenue_12mo'],
axis=1
)

In [120]:
# Merge with your model predictions: predictions_df with ['customer_ID', 'predicted_default']
revenue_df_test_2 = avg_df.merge(test2_pred, on='customer_ID', how='left')

# Apply 0 revenue for predicted defaulters
revenue_df_test_2['expected_revenue_12mo'] = revenue_df_test_2.apply(
    lambda row: 0 if row['scores'] == 1 else row['expected_revenue_12mo'],
axis=1
)

In [121]:
revenue_df_test_2

,customer_ID,S_Ave,B_Ave,monthly_revenue,expected_revenue_12mo,scores,target
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,0.006198,0.242507,0.004856,0.058276,NaN,NaN
1,000445609ff2a39d2dd02484899affa5696210a95f6869...,0.005166,0.814031,0.016286,0.195429,NaN,NaN
2,0004837f0c785928a29a6f83f70f4a1c54caec483a773f...,0.118214,0.051907,0.001156,0.013876,0.312187,0.0
3,0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704...,0.058062,1.007566,0.020209,0.242513,NaN,NaN
4,00050d84c6d26e26cd2b18c3eed83d3130c270e2361470...,0.059079,0.529799,0.010655,0.127861,NaN,NaN
...,...,...,...,...,...,...,...
91778,fffe2bc02423407e33a607660caeed076d713d8a5ad323...,0.783985,0.016944,0.001123,0.013474,NaN,NaN
91779,ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fd...,0.007299,0.048082,0.000969,0.011627,NaN,NaN
91780,ffff9984b999fccb2b6127635ed0736dda94e544e67e02...,0.027354,1.004100,0.020109,0.241312,0.001583,0.0
91781,ffffa5c46bc8de74f5a4554e74e239c8dee6b9baf38814...,0.018591,0.627542,0.012569,0.150833,NaN,NaN


In [122]:
# Define the range of thresholds to test
thresholds_to_test = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

results = []

for t in thresholds_to_test:
    dr, rev = evaluate_strategy(
        df=revenue_df,                    # or whatever your training set is
        target_col='target',
        prob_col='scores',
        balance_col='B_Ave',
        spend_col='S_Ave',
        threshold=t
    )
    results.append({'Threshold': t, 'Default Rate (%)': dr * 100, 'Expected Revenue': rev})

results_df = pd.DataFrame(results)
print(results_df.sort_values('Threshold'))

C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['monthly_revenue'] = accepted[balance_col] * 0.02 + accepted[spend_col] * 0.001
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

   Threshold  Default Rate (%)  Expected Revenue
0        0.1              0.39           7450.53
1        0.2              1.09           7860.20
2        0.3              2.02           8068.59
3        0.4              3.29           8192.81
4        0.5              5.01           8272.02
5        0.6              7.12           8320.14
6        0.7              9.88           8346.93
7        0.8             13.43           8358.22
8        0.9             17.82           8360.50
9        1.0             25.77           8360.56


C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(


In [123]:
# Define the range of thresholds to test
thresholds_to_test = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

results = []

for t in thresholds_to_test:
    dr, rev = evaluate_strategy(
        df=revenue_df_test_1,                    # or whatever your training set is
        target_col='target',
        prob_col='scores',
        balance_col='B_Ave',
        spend_col='S_Ave',
        threshold=t
    )
    results.append({'Threshold': t, 'Default Rate (%)': dr * 100, 'Expected Revenue': rev})

results_df = pd.DataFrame(results)
print(results_df.sort_values('Threshold'))

C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['monthly_revenue'] = accepted[balance_col] * 0.02 + accepted[spend_col] * 0.001
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

   Threshold  Default Rate (%)  Expected Revenue
0        0.1              1.80           1567.76
1        0.2              3.27           1645.24
2        0.3              4.83           1690.83
3        0.4              6.28           1719.97
4        0.5              8.14           1741.77
5        0.6             10.01           1757.13
6        0.7             12.54           1767.63
7        0.8             15.56           1776.44
8        0.9             18.88           1782.76
9        1.0             25.71           1786.56


C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(


In [124]:
# Define the range of thresholds to test
thresholds_to_test = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]

results = []

for t in thresholds_to_test:
    dr, rev = evaluate_strategy(
        df=revenue_df_test_2,                    # or whatever your training set is
        target_col='target',
        prob_col='scores',
        balance_col='B_Ave',
        spend_col='S_Ave',
        threshold=t
    )
    results.append({'Threshold': t, 'Default Rate (%)': dr * 100, 'Expected Revenue': rev})

results_df = pd.DataFrame(results)
print(results_df.sort_values('Threshold'))

C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['monthly_revenue'] = accepted[balance_col] * 0.02 + accepted[spend_col] * 0.001
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

   Threshold  Default Rate (%)  Expected Revenue
0        0.1              1.90           1595.90
1        0.2              3.28           1677.35
2        0.3              4.84           1727.71
3        0.4              6.70           1750.91
4        0.5              8.50           1774.52
5        0.6             10.36           1789.15
6        0.7             12.78           1799.48
7        0.8             15.50           1805.83
8        0.9             18.78           1809.97
9        1.0             25.04           1813.01


C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['monthly_revenue'] = accepted[balance_col] * 0.02 + accepted[spend_col] * 0.001
C:\Users\shrey\AppData\Local\Temp\ipykernel_10880\942672650.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  accepted['expected_12mo_revenue'] = accepted.apply(
